In [15]:
# Basic
import gym
import numpy as np
import pandas as pd
import os
import sys
import pickle
from time import sleep
from collections import namedtuple
from typing import List, Dict, NoReturn, Tuple, Optional, Any

# Visualization pretty
from pprint import pprint
from tqdm.notebook import tqdm_notebook
from IPython.display import clear_output

# NNs
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## Environment

**Description:** <br>
        A pole is attached by an un-actuated joint to a cart, which moves along
        a frictionless track. The pendulum starts upright, and the goal is to
        prevent it from falling over by increasing and reducing the cart's
        velocity.
        
**Source:** <br>
        This environment corresponds to the version of the cart-pole problem
        described by Barto, Sutton, and Anderson
        
**Observation:** <br>
        Type: Box(4) <br>
Num    |  Observation             |        Min             |       Max
* 0    |   Cart Position          |   -4.8                 |   4.8
* 1    |   Cart Velocity          |  -Inf                  |  Inf
* 2    |   Pole Angle             |   -0.418 rad (-24 deg) |   0.418 rad (24 deg)
* 3    |   Pole Angular Velocity  |   -Inf                 |  Inf
        
**Actions:** <br>
        Type: Discrete(2) <br>
        Num   Action <br>
* 0     Push cart to the left
* 1     Push cart to the right

> Note: The amount the velocity that is reduced or increased is not
        fixed; it depends on the angle the pole is pointing. This is because
        the center of gravity of the pole increases the amount of energy needed
        to move the cart underneath it
        
**Reward:** <br>
        Reward is 1 for every step taken, including the termination step
        
**Starting State:** <br>
        All observations are assigned a uniform random value in [-0.05..0.05]
        
**Episode Termination:** <br>
* Pole Angle is more than 12 degrees.
* Cart Position is more than 2.4 (center of the cart reaches the edge of the display).
* Episode length is greater than 200.
* Solved Requirements:
* Considered solved when the average return is greater than or equal to 195.0 over 100 consecutive trials.

In [11]:
# loading Cartpole environment from gym
env = gym.make('CartPole-v1')
env.seed(seed=11)
print(f"Action space: {env.action_space.n}")  
# Cart Position, Cart Velocity, Pole Angle, Pole Angular Velocity
print(f"Observation space: shape {env.observation_space.shape},\n{env.observation_space.low} to {env.observation_space.high}")

Action space: 2
Observation space: shape (4,),
[-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38] to [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]


In [10]:
try:
#     env = gym.wrappers.Monitor(env, "./gym-results", force=True)
    env.reset()
    for _ in range(200):
        env.render(mode='human')
        clear_output(wait=True)
        env.step(env.action_space.sample()) # take a random action
finally:
    env.close()

## Agent

In [14]:
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])

In [ ]:
class CEM_Agent(nn.Module):
    
    def __init__(self, env, h_size=16):
        super(CEM_Agent, self).__init__()
        self._env = env
        # state, hidden layer, action sizes
        self._state_size = env.observation_space.shape[0]
        self._hidd_size = h_size
        self._act_size = env.action_space.shape[0]
        
        # define net
        self._net = nn.Sequential(
                nn.Linear(self._state_size, self._hidd_size),
                nn.ReLU(),
                nn.Linear(self._hidd_size, self._act_size))
        

In [17]:
net = nn.Sequential(
                nn.Linear(env.observation_space.shape[0], 16),
                nn.ReLU(),
                nn.Linear(16, env.action_space.n))
net

Sequential(
  (0): Linear(in_features=4, out_features=16, bias=True)
  (1): ReLU()
  (2): Linear(in_features=16, out_features=2, bias=True)
)

In [36]:
for i, c in net.named_children():
#     print(c._parameters.)
    if len(c._parameters.keys()) > 0:
        print(c._parameters.get('weight').data)
        print(c._parameters.get('bias').data)

tensor([[-0.4678, -0.2683,  0.3372,  0.3456],
        [-0.0756,  0.0177, -0.4375,  0.3075],
        [ 0.3758, -0.2152,  0.4257, -0.1055],
        [-0.2111,  0.4944,  0.4736,  0.1178],
        [ 0.4323, -0.1353,  0.2952, -0.0540],
        [ 0.0660, -0.3630, -0.4889, -0.2233],
        [ 0.1217, -0.0835,  0.1415,  0.0761],
        [ 0.1484,  0.3977, -0.1751,  0.3275],
        [-0.0831,  0.3808, -0.2992, -0.0203],
        [ 0.4031, -0.1227, -0.4576, -0.3107],
        [-0.4232, -0.1561,  0.0227,  0.0544],
        [-0.0597,  0.1380,  0.0657, -0.1194],
        [-0.4617,  0.3804, -0.3038, -0.4704],
        [ 0.2972,  0.2881,  0.2758, -0.0191],
        [ 0.2768, -0.3381, -0.2655,  0.4913],
        [ 0.0468,  0.2630, -0.2954, -0.0433]])
tensor([ 0.2263, -0.3369,  0.1547,  0.0916,  0.4281,  0.0040, -0.1069,  0.3983,
         0.1310,  0.1471, -0.0054, -0.1154, -0.0494,  0.1611,  0.4549,  0.1091])
tensor([[ 0.0664,  0.2487,  0.2020, -0.2046,  0.1665,  0.2033,  0.1399, -0.0641,
          0.0558,  0.